In [18]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [19]:
load_dotenv()

llm = ChatOpenAI()

In [20]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [21]:
def generate_joke(state: JokeState):
    prompt = f'generate a joke on the topic {state['topic']}'
    response = llm.invoke(prompt).content
    
    return {'joke': response}
    

In [22]:
def generate_explanation(state: JokeState):
    prompt = f'write an explaination for the joke - {state['joke']}'
    
    response = llm.invoke(prompt).content
    
    return {'explanation' : response}

In [30]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [37]:
config0 = {'configurable': {"thread_id":"0"}}
workflow.invoke({'topic':'pizza'}, config=config0)

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the party? \nBecause it wanted to get a "pizza" the action!',
 'explanation': 'This joke is a play on words - the phrase "get a piece of the action" is a common expression that means to get involved or participate in something exciting or enjoyable. In this joke, the pizza went to the party because it wanted to "get a \'pizza\' of the action," with "pizza" sounding like "piece of." It\'s a light-hearted and punny explanation for why the pizza decided to attend the party.'}

In [38]:
workflow.get_state(config0)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party? \nBecause it wanted to get a "pizza" the action!', 'explanation': 'This joke is a play on words - the phrase "get a piece of the action" is a common expression that means to get involved or participate in something exciting or enjoyable. In this joke, the pizza went to the party because it wanted to "get a \'pizza\' of the action," with "pizza" sounding like "piece of." It\'s a light-hearted and punny explanation for why the pizza decided to attend the party.'}, next=(), config={'configurable': {'thread_id': '0', 'checkpoint_ns': '', 'checkpoint_id': '1f09aaa9-cdaf-6c96-8002-a7e49abd6e7f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-26T07:30:01.520014+00:00', parent_config={'configurable': {'thread_id': '0', 'checkpoint_ns': '', 'checkpoint_id': '1f09aaa9-c2f2-65fc-8001-67142e493cff'}}, tasks=(), interrupts=())

In [39]:
list(workflow.get_state_history(config0))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party? \nBecause it wanted to get a "pizza" the action!', 'explanation': 'This joke is a play on words - the phrase "get a piece of the action" is a common expression that means to get involved or participate in something exciting or enjoyable. In this joke, the pizza went to the party because it wanted to "get a \'pizza\' of the action," with "pizza" sounding like "piece of." It\'s a light-hearted and punny explanation for why the pizza decided to attend the party.'}, next=(), config={'configurable': {'thread_id': '0', 'checkpoint_ns': '', 'checkpoint_id': '1f09aaa9-cdaf-6c96-8002-a7e49abd6e7f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-26T07:30:01.520014+00:00', parent_config={'configurable': {'thread_id': '0', 'checkpoint_ns': '', 'checkpoint_id': '1f09aaa9-c2f2-65fc-8001-67142e493cff'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did 

In [41]:
config3 = {'configurable': {"thread_id":"3"}}
workflow.invoke({'topic':'pasta'}, config=config3)

{'topic': 'pasta',
 'joke': "Why did the pasta go to the party? \nBecause it didn't want to be left unpenne-d!",
 'explanation': 'This joke is a play on words using the pun "penne" which sounds similar to the word "alone". In this case, the pasta went to the party because it didn\'t want to be left "unpenne-d" or "un-penneed" which sounds like "un-penned" or "un-panned". It\'s a light-hearted and humorous way to make a joke about pasta attending a party to not be left alone.'}

### Time Travel

In [42]:
workflow.get_state({"configurable": {"thread_id":"0", "checkpoint_id":"1f09aaa9-bbb1-6a72-8000-4977c87bf7bb"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '0', 'checkpoint_id': '1f09aaa9-bbb1-6a72-8000-4977c87bf7bb'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-26T07:29:59.633347+00:00', parent_config={'configurable': {'thread_id': '0', 'checkpoint_ns': '', 'checkpoint_id': '1f09aaa9-bbae-61e4-bfff-c7611a74fcd7'}}, tasks=(PregelTask(id='24000527-5ae4-294d-b1a6-cb32a7dda43d', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza go to the party? \nBecause it wanted to get a "pizza" the action!'}),), interrupts=())

In [43]:
workflow.invoke(None, {"configurable": {"thread_id":"0", "checkpoint_id":"1f09aaa9-bbb1-6a72-8000-4977c87bf7bb"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the therapist? Because it wanted to get to the crust of its problems!',
 'explanation': 'This joke is a play on words, as "get to the crust of the problem" is a common saying that means to get to the root or core of an issue. In this joke, the pizza goes to the therapist not because it has emotional problems like a human would, but because it literally wants to get to the crust, or the outer edge, of its problems - in this case, the crust of the pizza itself. The humor comes from the absurdity of a pizza seeking therapy for its food-related issues.'}

In [44]:
list(workflow.get_state_history(config0))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the therapist? Because it wanted to get to the crust of its problems!', 'explanation': 'This joke is a play on words, as "get to the crust of the problem" is a common saying that means to get to the root or core of an issue. In this joke, the pizza goes to the therapist not because it has emotional problems like a human would, but because it literally wants to get to the crust, or the outer edge, of its problems - in this case, the crust of the pizza itself. The humor comes from the absurdity of a pizza seeking therapy for its food-related issues.'}, next=(), config={'configurable': {'thread_id': '0', 'checkpoint_ns': '', 'checkpoint_id': '1f09ab7a-482f-6272-8002-5e133143502e'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-26T09:03:17.822297+00:00', parent_config={'configurable': {'thread_id': '0', 'checkpoint_ns': '', 'checkpoint_id': '1f09ab7a-34a1-649b-8001-7a41f4ffc28a'}}, tasks